In [7]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split
!pip install torchattacks
import torchattacks
!pip install torchinfo
from torchinfo import summary


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),                                            
            nn.MaxPool2d(kernel_size=2),   
            nn.ReLU(),         
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),                        
            nn.MaxPool2d(2),   
            nn.ReLU(),                
        )
        self.out = nn.Linear(1152, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model = Network()
print(repr(model) )
print(model)
summary(model, input_size=(16, 1, 28, 28))

Network(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (out): Linear(in_features=1152, out_features=10, bias=True)
)
Network(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (out): Linear(in_features=1152, out_features=10, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
Network                                  [16, 10]                  --
├─Sequential: 1-1                        [16, 16, 12, 12]          --
│    └─Conv2d: 2-1                       [16, 16, 24, 24]          416
│    └─MaxPool2d: 2-2                    [16, 16, 12, 12]          --
│    └─ReLU: 2-3                         [16, 16, 12, 12]          --
├─Sequential: 1-2                        [16, 32, 6, 6]            --
│    └─Conv2d: 2-4                       [16, 32, 12, 12]          12,832
│    └─MaxPool2d: 2-5                    [16, 32, 6, 6]            --
│    └─ReLU: 2-6                         [16, 32, 6, 6]            --
├─Linear: 1-3                            [16, 10]                  11,530
Total params: 24,778
Trainable params: 24,778
Non-trainable params: 0
Total mult-adds (M): 33.58
Input size (MB): 0.05
Forward/backward pass size (MB): 1.77
Params size (MB): 0.10
Estimated Total Size (MB): 1.92